In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from prophet import Prophet
from tqdm import tqdm
import pickle
import os
from datetime import datetime
import json

# Define consistent colors for plots
TRAIN_COLOR = 'steelblue'
TRAIN_FILL_COLOR = 'steelblue'
TRAIN_FILL_ALPHA = 0.3
VAL_COLOR = 'coral'
VAL_FILL_COLOR = 'coral'
VAL_FILL_ALPHA = 0.3
TEST_COLOR = 'forestgreen'
TEST_FILL_COLOR = 'forestgreen'
TEST_FILL_ALPHA = 0.3

# Set random seeds for reproducibility
def set_seeds(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    
set_seeds()

## Data Loading and Preprocessing

In [2]:
def load_data(file_path):
    data = pd.read_parquet(file_path)
    return data

def check_for_missing_values(data):
    missing_values = data.isnull().sum()
    if missing_values.any():
        print("Missing values found in the dataset:")
        print(missing_values[missing_values > 0])
    else:
        print("No missing values found in the dataset.")
    return missing_values

def split_data(data, train_years, val_year, test_year):
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    
    train_data = data[data['timestamp'].dt.year.isin(train_years)]
    val_data = data[data['timestamp'].dt.year == val_year]
    test_data = data[data['timestamp'].dt.year == test_year]
    
    print(f"Train data size: {len(train_data)}")
    print(f"Validation data size: {len(val_data)}")
    print(f"Test data size: {len(test_data)}")
    
    return train_data, val_data, test_data

def filter_ride_data(data, ride_name):
    return data[data[f'ride_name_{ride_name}'] == True].copy()

def get_all_rides(data):
    ride_columns = [col for col in data.columns if col.startswith('ride_name_')]
    return [col.replace('ride_name_', '') for col in ride_columns]

def filter_to_operating_hours(ride_data):
    # Determine operating hours from data where wait times > 0
    operating_hours = ride_data[ride_data["wait_time"] > 0].groupby(
        ride_data["timestamp"].dt.date
    )["timestamp"].agg(['min', 'max']).reset_index()
    
    # Extract opening and closing hours
    operating_hours['opening_hour'] = pd.to_datetime(operating_hours['min']).dt.hour
    operating_hours['closing_hour'] = pd.to_datetime(operating_hours['max']).dt.hour
    
    # Set reasonable boundaries for operating hours
    operating_hours['opening_hour'] = operating_hours['opening_hour'].clip(lower=9, upper=11)
    operating_hours['closing_hour'] = operating_hours['closing_hour'].clip(lower=17, upper=21)
    
    # Create date-to-hours mapping
    date_to_hours = {}
    for _, row in operating_hours.iterrows():
        date_to_hours[row['timestamp']] = (row['opening_hour'], row['closing_hour'])
    
    # Filter data to operating hours only
    def is_operating_hour(timestamp):
        date = timestamp.date()
        if date not in date_to_hours:
            return 0
        
        open_hour, close_hour = date_to_hours[date]
        hour = timestamp.hour
        return 1 if open_hour <= hour < close_hour else 0
    
    ride_data['operating_hour'] = ride_data['timestamp'].apply(is_operating_hour)
    ride_data = ride_data[ride_data['operating_hour'] == 1]
    ride_data = ride_data.drop(columns=["operating_hour"])
    
    return ride_data

## Time Series Decomposition with Prophet

In [3]:
class BaseTimeSeriesModel:
    def __init__(self):
        self.model = None
        self.forecast = None
        self.holidays = None
        
    def prepare_prophet_dataframe(self, data, include_y=True):
        prophet_df = data[['timestamp', 'wait_time', 'temperature_unscaled', 'rain_unscaled', 'is_weekend']].copy()
        prophet_df = prophet_df.rename(columns={'timestamp': 'ds', 'wait_time': 'y', 'temperature_unscaled': 'temperature', 'rain_unscaled': 'rain'})

        if not include_y:
            prophet_df = prophet_df.drop(["y"], axis=1) 
        
        # Add additional features
        prophet_df['temp_squared'] = prophet_df['temperature'] ** 2
        prophet_df['high_temp'] = (prophet_df['temperature'] > 25).astype(int)
        prophet_df['any_rain'] = (prophet_df['rain'] > 0).astype(int)
        prophet_df['temp_weekend'] = prophet_df['temperature'] * prophet_df['is_weekend']
        prophet_df['rain_weekend'] = prophet_df['rain'] * prophet_df['is_weekend']
        
        return prophet_df
    
    def create_holiday_dataframes(self, data):
        holiday_dfs = []
        
        # Process country holidays
        for country in ['swiss', 'german', 'french']:
            holiday_col = f"is_{country}_holiday"
            if holiday_col in data.columns:
                country_holidays = data.loc[data[holiday_col] == 1, ["timestamp"]]
                if len(country_holidays) > 0:
                    country_holidays["ds"] = pd.to_datetime(country_holidays["timestamp"]).dt.date
                    country_holidays = country_holidays.drop_duplicates(subset=["ds"])
                    country_holidays = country_holidays.drop(columns=["timestamp"])
                    country_holidays["holiday"] = f"{country}_holiday"
                    country_holidays["lower_window"] = 0
                    country_holidays["upper_window"] = 0
                    holiday_dfs.append(country_holidays.reset_index(drop=True))
        # Combine all holidays
        if holiday_dfs:
            all_holidays = pd.concat(holiday_dfs)
            all_holidays["ds"] = pd.to_datetime(all_holidays["ds"])
            return all_holidays.sort_values(by=["ds"]).reset_index(drop=True)
        return None
    
    def fit(self, prophet_df):
        # Create holidays dataframe
        self.holidays = self.create_holiday_dataframes(prophet_df)
        
        # Create a Prophet model with configuration from prophet_model.py
        self.model = Prophet(
            # Core parameters
            seasonality_mode='multiplicative',  # Better for tourism/attraction data
            changepoint_prior_scale=0.05,       # Flexibility in trend changes
            changepoint_range=0.95,             # Allow changepoints closer to the end

            # Handling seasonality
            seasonality_prior_scale=12,         # Stronger seasonality influence
            yearly_seasonality=True,            # Capture yearly patterns
            weekly_seasonality=True,            # Capture weekly patterns
            daily_seasonality=True,             # Custom daily seasonality for operating hours

            # Specific to park operation
            holidays_prior_scale=15,            # Strong holiday effects for parks
            holidays=self.holidays,             # Include holidays and covid periods
            interval_width=0.95                 # 95% confidence interval
        )

        # Add weather and interaction regressors
        self.model.add_regressor('temperature', mode='multiplicative', standardize=True)
        self.model.add_regressor('rain', mode='multiplicative', standardize=True)
        self.model.add_regressor('temp_squared', mode='additive', standardize=True)
        self.model.add_regressor('high_temp', mode='multiplicative', standardize=False)
        self.model.add_regressor('any_rain', mode='multiplicative', standardize=False)
        self.model.add_regressor('temp_weekend', mode='additive', standardize=True)
        self.model.add_regressor('rain_weekend', mode='multiplicative', standardize=True)
        
        prophet_df['between_covid_lockdowns'] = 0
        covid_period = (prophet_df['ds'] >= '2020-05-20') & (prophet_df['ds'] <= '2020-11-01')
        prophet_df.loc[covid_period, 'between_covid_lockdowns'] = 1
        self.model.add_regressor('between_covid_lockdowns')
        
        prophet_df['covid_recovery'] = 0
        recovery_period = (prophet_df['ds'] >= '2021-05-21') & (prophet_df['ds'] <= '2021-08-31')
        prophet_df.loc[recovery_period, 'covid_recovery'] = 1
        self.model.add_regressor('covid_recovery')

        self.model.fit(prophet_df)
        return self.model

    def predict(self, future_df):
        # Add required columns for prediction if they're not already present
        future_df = future_df.copy()
        
        # Add COVID regressors if they're not already present
        if 'between_covid_lockdowns' not in future_df.columns:
            future_df['between_covid_lockdowns'] = 0
            covid_period = (future_df['ds'] >= '2020-05-20') & (future_df['ds'] <= '2020-11-01')
            future_df.loc[covid_period, 'between_covid_lockdowns'] = 1
            
        if 'covid_recovery' not in future_df.columns:
            future_df['covid_recovery'] = 0
            recovery_period = (future_df['ds'] >= '2021-05-21') & (future_df['ds'] <= '2021-08-31')
            future_df.loc[recovery_period, 'covid_recovery'] = 1
        
        # Make predictions
        self.forecast = self.model.predict(future_df)
        
        # Apply post-processing
        self.forecast = self.post_process_forecast(self.forecast)
        
        return self.forecast
    
    def post_process_forecast(self, forecast):
        forecast = forecast.copy()
        
        # Correct negative predictions
        negative_mask = forecast['yhat'] < 0
        forecast.loc[negative_mask, 'yhat'] = 0
        forecast.loc[negative_mask, 'yhat_lower'] = 0
        forecast.loc[negative_mask, 'yhat_upper'] = 0
        
        return forecast
    
    def merge_predictions(self, original_data, forecast_data):
        result = original_data.copy()
        
        # Identify forecast columns to keep
        forecast_columns = ['ds', 'trend', 'yhat', 'yhat_lower', 'yhat_upper']
        for component in ['weekly', 'daily']:
            if component in forecast_data.columns:
                forecast_columns.append(component)
        
        result = pd.merge(
            result, 
            forecast_data[forecast_columns], 
            left_on='timestamp', 
            right_on='ds', 
            how='left'
        )
        
        result['residual'] = result['wait_time'] - result['yhat']
        return result

## Model Evaluation Functions

In [4]:
def evaluate_model(ride_df, actual_values, predictions, title=""):
    # Calculate metrics
    mae = np.mean(np.abs(predictions - actual_values))
    rmse = np.sqrt(np.mean(np.square(predictions - actual_values)))
    
    # For sMAPE, avoid division by zero
    epsilon = 1e-8
    abs_pct_errors = np.abs(predictions - actual_values) / (np.abs(predictions) + np.abs(actual_values) + epsilon)
    # Only include points where actual values are non-zero
    non_zero_mask = (actual_values > 0) & (predictions > 0)
    smape = np.mean(abs_pct_errors[non_zero_mask]) * 100 if np.any(non_zero_mask) else 0

    # Print metrics
    print(f"\n{title} MAE: {mae:.2f} minutes")
    print(f"{title} RMSE: {rmse:.2f} minutes")
    print(f"{title} sMAPE: {smape:.2f}%")
    
    # Create a DataFrame with results for time-based analysis
    results_df = pd.DataFrame({
        'timestamp': ride_df['timestamp'].values,
        'actual': actual_values,
        'predicted': predictions,
    })
    
    # Add time components
    results_df['hour'] = results_df['timestamp'].dt.hour
    results_df['day_of_week'] = results_df['timestamp'].dt.dayofweek
    results_df['month'] = results_df['timestamp'].dt.month
    
    # Calculate errors
    results_df['error'] = results_df['predicted'] - results_df['actual']
    results_df['abs_error'] = np.abs(results_df['error'])
    results_df['pct_error'] = abs_pct_errors * 100
    
    # Create metrics dictionary
    metrics = {
        "mae": mae,
        "rmse": rmse,
        "smape": smape
    }
    
    return metrics, results_df

def create_evaluation_plots(results_df, title=""):
    """Create evaluation plots for model performance"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Actual vs Predicted scatter plot
    axes[0, 0].scatter(results_df['actual'], results_df['predicted'], alpha=0.5)
    max_val = max(results_df['actual'].max(), results_df['predicted'].max())
    axes[0, 0].plot([0, max_val], [0, max_val], 'k--')
    axes[0, 0].set_xlabel('Actual Wait Time (minutes)')
    axes[0, 0].set_ylabel('Predicted Wait Time (minutes)')
    axes[0, 0].set_title(f'{title} - Actual vs Predicted')
    
    # Hourly error analysis
    hourly_errors = results_df.groupby('hour')['abs_error'].mean()
    hourly_errors.plot(kind='bar', ax=axes[0, 1])
    axes[0, 1].set_xlabel('Hour of Day')
    axes[0, 1].set_ylabel('Mean Absolute Error (minutes)')
    axes[0, 1].set_title(f'{title} - Error by Hour')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Residual plot
    axes[1, 0].scatter(results_df['predicted'], results_df['error'], alpha=0.5)
    axes[1, 0].axhline(y=0, color='k', linestyle='--')
    axes[1, 0].set_xlabel('Predicted Wait Time (minutes)')
    axes[1, 0].set_ylabel('Residual (minutes)')
    axes[1, 0].set_title(f'{title} - Residual Plot')
    
    # Time series plot (sample)
    if len(results_df) > 1000:
        # Sample for visualization if too many points
        sample_df = results_df.sample(1000).sort_values('timestamp')
    else:
        sample_df = results_df.sort_values('timestamp')
    
    axes[1, 1].plot(sample_df['timestamp'], sample_df['actual'], label='Actual', alpha=0.7)
    axes[1, 1].plot(sample_df['timestamp'], sample_df['predicted'], label='Predicted', alpha=0.7)
    axes[1, 1].set_xlabel('Time')
    axes[1, 1].set_ylabel('Wait Time (minutes)')
    axes[1, 1].set_title(f'{title} - Time Series (Sample)')
    axes[1, 1].legend()
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    return fig

## Model Storage Functions

In [5]:
def save_model(ride_name, prophet_model, metrics, output_dir="models"):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create ride-specific directory
    ride_dir = os.path.join(output_dir, ride_name.replace(" ", "_"))
    os.makedirs(ride_dir, exist_ok=True)
    
    # Save Prophet model (using pickle)
    with open(os.path.join(ride_dir, "prophet_model.pkl"), "wb") as f:
        pickle.dump(prophet_model.model, f)
    
    # Save holidays if they exist
    if prophet_model.holidays is not None:
        prophet_model.holidays.to_csv(os.path.join(ride_dir, "holidays.csv"), index=False)
    
    # Save metrics
    with open(os.path.join(ride_dir, "metrics.json"), "w") as f:
        json.dump(metrics, f, indent=4)
    
    print(f"Models and results saved to {ride_dir}")

def load_model(ride_name, output_dir="models"):
    # Create ride-specific directory path
    ride_dir = os.path.join(output_dir, ride_name.replace(" ", "_"))
    
    # Check if models exist
    if not os.path.exists(ride_dir):
        return None, None
    
    # Load Prophet model
    try:
        with open(os.path.join(ride_dir, "prophet_model.pkl"), "rb") as f:
            prophet_model_obj = pickle.load(f)
    except:
        print(f"Could not load model for {ride_name}")
        return None, None
    
    # Initialize BaseTimeSeriesModel and set the loaded model
    prophet_ts = BaseTimeSeriesModel()
    prophet_ts.model = prophet_model_obj
    
    # Load holidays if they exist
    if os.path.exists(os.path.join(ride_dir, "holidays.csv")):
        prophet_ts.holidays = pd.read_csv(os.path.join(ride_dir, "holidays.csv"))
    
    # Load metrics
    with open(os.path.join(ride_dir, "metrics.json"), "r") as f:
        metrics = json.load(f)
    
    return prophet_ts, metrics

def get_processed_rides(output_dir="models"):
    if not os.path.exists(output_dir):
        return []
    
    # Get all subdirectories in the output directory
    processed_rides = [d for d in os.listdir(output_dir) 
                      if os.path.isdir(os.path.join(output_dir, d))]
    
    # Convert directory names back to ride names
    processed_rides = [ride.replace("_", " ") for ride in processed_rides]
    
    return processed_rides

## Multi-Ride Processing

In [6]:
def create_checkpoint_file(processed_rides, output_dir="models"):
    checkpoint_path = os.path.join(output_dir, "checkpoint.json")
    with open(checkpoint_path, "w") as f:
        json.dump({"processed_rides": processed_rides}, f, indent=4)

def load_checkpoint_file(output_dir="models"):
    checkpoint_path = os.path.join(output_dir, "checkpoint.json")
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, "r") as f:
            checkpoint = json.load(f)
        return checkpoint.get("processed_rides", [])
    return []

## Training Pipeline for Single Ride (Extended with Test Set)

In [7]:
def process_single_ride_extended(ride_name, train_data, val_data, test_data, output_dir="models"):
    """Process a single ride with both validation and test set evaluation."""
    print(f"\n{'='*50}")
    print(f"Processing ride: {ride_name}")
    print(f"{'='*50}")
    
    # Filter data for the current ride
    ride_train_data = filter_ride_data(train_data, ride_name)
    ride_val_data = filter_ride_data(val_data, ride_name)
    ride_test_data = filter_ride_data(test_data, ride_name)
    
    print(f"Training data size: {len(ride_train_data)}")
    print(f"Validation data size: {len(ride_val_data)}")
    print(f"Test data size: {len(ride_test_data)}")
    
    # Skip if not enough data
    if len(ride_train_data) < 100 or len(ride_val_data) < 50 or len(ride_test_data) < 50:
        print(f"Skipping {ride_name} due to insufficient data")
        return None
    
    # VALIDATION EVALUATION: Train on train data only, evaluate on validation set
    print("Training Prophet model on train data for validation evaluation...")
    prophet_val = BaseTimeSeriesModel()
    prophet_val_df = prophet_val.prepare_prophet_dataframe(ride_train_data)
    prophet_val.fit(prophet_val_df)
    
    # Generate forecasts for validation data
    print("Evaluating on validation set...")
    future_val = prophet_val.prepare_prophet_dataframe(ride_val_data, include_y=False)
    val_forecast = prophet_val.predict(future_val)
    
    # Merge predictions with validation data
    ride_val_data_with_forecast = pd.merge(
        ride_val_data,
        val_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']],
        left_on='timestamp', 
        right_on='ds',
        how='left'
    )
    
    # Evaluate on validation set
    val_actuals = ride_val_data_with_forecast['wait_time'].values
    val_predictions = ride_val_data_with_forecast['yhat'].values
    val_metrics, val_results_df = evaluate_model(
        ride_val_data_with_forecast, val_actuals, val_predictions, 
        title=f"{ride_name} - Validation"
    )
    
    # TEST EVALUATION: Train on combined train + validation data, evaluate on test set
    print("Training Prophet model on combined train + validation data for test evaluation...")
    combined_train_val = pd.concat([ride_train_data, ride_val_data], ignore_index=True)
    prophet_test = BaseTimeSeriesModel()
    prophet_test_df = prophet_test.prepare_prophet_dataframe(combined_train_val)
    prophet_test.fit(prophet_test_df)
    
    # Generate forecasts for test data
    print("Evaluating on test set...")
    future_test = prophet_test.prepare_prophet_dataframe(ride_test_data, include_y=False)
    test_forecast = prophet_test.predict(future_test)
    
    # Merge predictions with test data
    ride_test_data_with_forecast = pd.merge(
        ride_test_data,
        test_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']],
        left_on='timestamp', 
        right_on='ds',
        how='left'
    )
    
    # Evaluate on test set
    test_actuals = ride_test_data_with_forecast['wait_time'].values
    test_predictions = ride_test_data_with_forecast['yhat'].values
    test_metrics, test_results_df = evaluate_model(
        ride_test_data_with_forecast, test_actuals, test_predictions, 
        title=f"{ride_name} - Test"
    )
    
    # Create figures directory
    fig_dir = os.path.join(output_dir, ride_name.replace(" ", "_"), "figures")
    os.makedirs(fig_dir, exist_ok=True)
    
    # Save model components visualization (using test model which is trained on more data)
    fig_components = prophet_test.model.plot_components(test_forecast, figsize=(14, 10))
    fig_components.savefig(os.path.join(fig_dir, "model_components.png"))
    plt.close(fig_components)

    # Save evaluation figures
    fig_val_eval = create_evaluation_plots(val_results_df, f"{ride_name} - Validation")
    fig_val_eval.savefig(os.path.join(fig_dir, "validation_evaluation.png"))
    plt.close(fig_val_eval)
    
    fig_test_eval = create_evaluation_plots(test_results_df, f"{ride_name} - Test")
    fig_test_eval.savefig(os.path.join(fig_dir, "test_evaluation.png"))
    plt.close(fig_test_eval)
    
    # Prepare combined metrics
    combined_metrics = {
        "validation": val_metrics,
        "test": test_metrics,
        "data_counts": {
            "train": len(ride_train_data),
            "validation": len(ride_val_data),
            "test": len(ride_test_data),
            "combined_train_val": len(combined_train_val)
        },
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Save models and results (save the test model which is trained on more data)
    save_model(ride_name, prophet_test, combined_metrics, output_dir)
    
    # Save detailed results as CSV
    results_dir = os.path.join(output_dir, ride_name.replace(" ", "_"), "results")
    os.makedirs(results_dir, exist_ok=True)
    val_results_df.to_csv(os.path.join(results_dir, "validation_results.csv"), index=False)
    test_results_df.to_csv(os.path.join(results_dir, "test_results.csv"), index=False)
    
    return combined_metrics

## Training Pipeline for All Rides (Extended)

In [8]:
def generate_summary_report_extended(all_results, output_dir="models"):
    """Generate comprehensive summary report for both validation and test sets."""
    # Create lists to store summary data
    summary_data = []
    
    # Extract data from results
    for ride_name, metrics in all_results.items():
        if not metrics:
            continue
        
        # Base ride info
        base_info = {
            "ride_name": ride_name,
            "train_data_size": metrics.get("data_counts", {}).get("train", 0),
            "val_data_size": metrics.get("data_counts", {}).get("validation", 0),
            "test_data_size": metrics.get("data_counts", {}).get("test", 0),
            "combined_train_val_size": metrics.get("data_counts", {}).get("combined_train_val", 0)
        }
        
        # Add validation metrics
        if "validation" in metrics:
            val_row = base_info.copy()
            val_row["dataset"] = "validation"
            val_metrics = metrics["validation"]
            val_row["mae"] = val_metrics.get("mae", float('nan'))
            val_row["rmse"] = val_metrics.get("rmse", float('nan'))
            val_row["smape"] = val_metrics.get("smape", float('nan'))
            summary_data.append(val_row)
        
        # Add test metrics
        if "test" in metrics:
            test_row = base_info.copy()
            test_row["dataset"] = "test"
            test_metrics = metrics["test"]
            test_row["mae"] = test_metrics.get("mae", float('nan'))
            test_row["rmse"] = test_metrics.get("rmse", float('nan'))
            test_row["smape"] = test_metrics.get("smape", float('nan'))
            summary_data.append(test_row)
    
    # Create DataFrame
    detailed_summary = pd.DataFrame(summary_data)
    
    if len(detailed_summary) == 0:
        print("No results to summarize.")
        return None
    
    # Save detailed summary
    detailed_path = os.path.join(output_dir, "detailed_prophet_summary_extended.csv")
    detailed_summary.to_csv(detailed_path, index=False)
    
    # Create aggregated summary for each dataset
    val_summary = detailed_summary[detailed_summary['dataset'] == 'validation'].groupby('dataset').agg({
        'mae': ['mean', 'std', 'median'],
        'rmse': ['mean', 'std', 'median'],
        'smape': ['mean', 'std', 'median']
    }).round(2)
    
    test_summary = detailed_summary[detailed_summary['dataset'] == 'test'].groupby('dataset').agg({
        'mae': ['mean', 'std', 'median'],
        'rmse': ['mean', 'std', 'median'],
        'smape': ['mean', 'std', 'median']
    }).round(2)
    
    # Create simple summary DataFrame for comparison
    simple_summary = []
    
    val_data = detailed_summary[detailed_summary['dataset'] == 'validation']
    test_data = detailed_summary[detailed_summary['dataset'] == 'test']
    
    if len(val_data) > 0:
        simple_summary.append({
            'dataset': 'validation',
            'count': len(val_data),
            'mae_mean': val_data['mae'].mean(),
            'mae_std': val_data['mae'].std(),
            'rmse_mean': val_data['rmse'].mean(),
            'rmse_std': val_data['rmse'].std(),
            'smape_mean': val_data['smape'].mean(),
            'smape_std': val_data['smape'].std()
        })
    
    if len(test_data) > 0:
        simple_summary.append({
            'dataset': 'test',
            'count': len(test_data),
            'mae_mean': test_data['mae'].mean(),
            'mae_std': test_data['mae'].std(),
            'rmse_mean': test_data['rmse'].mean(),
            'rmse_std': test_data['rmse'].std(),
            'smape_mean': test_data['smape'].mean(),
            'smape_std': test_data['smape'].std()
        })
    
    simple_summary_df = pd.DataFrame(simple_summary)
    
    # Save simple summary
    simple_path = os.path.join(output_dir, "prophet_summary_extended.csv")
    simple_summary_df.to_csv(simple_path, index=False)
    
    # Print summary statistics
    print("\n" + "="*80)
    print("PROPHET MODEL SUMMARY (VALIDATION & TEST SETS):")
    print(f"Total rides processed: {detailed_summary['ride_name'].nunique()}")
    
    if len(val_data) > 0:
        print(f"\nValidation Set Performance:")
        print(f"  Average MAE: {val_data['mae'].mean():.2f} ± {val_data['mae'].std():.2f}")
        print(f"  Average RMSE: {val_data['rmse'].mean():.2f} ± {val_data['rmse'].std():.2f}")
        print(f"  Average sMAPE: {val_data['smape'].mean():.2f}% ± {val_data['smape'].std():.2f}%")
    
    if len(test_data) > 0:
        print(f"\nTest Set Performance:")
        print(f"  Average MAE: {test_data['mae'].mean():.2f} ± {test_data['mae'].std():.2f}")
        print(f"  Average RMSE: {test_data['rmse'].mean():.2f} ± {test_data['rmse'].std():.2f}")
        print(f"  Average sMAPE: {test_data['smape'].mean():.2f}% ± {test_data['smape'].std():.2f}%")
    
    print(f"\nDetailed summary saved to: {detailed_path}")
    print(f"Simple summary saved to: {simple_path}")
    print("="*80)
    
    # Create visualizations
    create_prophet_visualizations_extended(detailed_summary, simple_summary_df, output_dir)
    
    return detailed_summary, simple_summary_df

def create_prophet_visualizations_extended(detailed_summary, simple_summary_df, output_dir):
    """Create visualizations for Prophet model results including both validation and test sets."""
    
    # 1. Performance comparison (validation vs test)
    if len(simple_summary_df) == 2:
        plt.figure(figsize=(15, 8))
        
        datasets = simple_summary_df['dataset']
        mae_means = simple_summary_df['mae_mean']
        mae_stds = simple_summary_df['mae_std']
        
        x = np.arange(len(datasets))
        width = 0.35
        
        bars = plt.bar(x, mae_means, width, yerr=mae_stds, capsize=5, 
                      color=[VAL_COLOR if d == 'validation' else TEST_COLOR for d in datasets],
                      alpha=0.8)
        
        # Add value labels on bars
        for bar, mean_val in zip(bars, mae_means):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + max(mae_means) * 0.01,
                    f'{mean_val:.2f}', ha='center', va='bottom', fontsize=16)
        
        plt.title('Prophet Model: Average MAE Comparison (Validation vs Test)', fontsize=20)
        plt.xlabel('Dataset', fontsize=14)
        plt.ylabel('Average MAE (minutes)', fontsize=14)
        plt.xticks(x, datasets)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "prophet_val_vs_test_comparison.png"))
        plt.savefig(os.path.join(output_dir, "prophet_val_vs_test_comparison.svg"))
        plt.close()
    
    # 2. Distribution comparison
    val_data = detailed_summary[detailed_summary['dataset'] == 'validation']
    test_data = detailed_summary[detailed_summary['dataset'] == 'test']
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    # MAE distribution
    if len(val_data) > 0:
        axes[0].hist(val_data['mae'], bins=15, alpha=0.7, label='Validation', color=VAL_COLOR)
    if len(test_data) > 0:
        axes[0].hist(test_data['mae'], bins=15, alpha=0.7, label='Test', color=TEST_COLOR)
    axes[0].set_xlabel('MAE (minutes)')
    axes[0].set_ylabel('Number of Rides')
    axes[0].set_title('MAE Distribution')
    axes[0].legend()
    
    # RMSE distribution
    if len(val_data) > 0:
        axes[1].hist(val_data['rmse'], bins=15, alpha=0.7, label='Validation', color=VAL_COLOR)
    if len(test_data) > 0:
        axes[1].hist(test_data['rmse'], bins=15, alpha=0.7, label='Test', color=TEST_COLOR)
    axes[1].set_xlabel('RMSE (minutes)')
    axes[1].set_ylabel('Number of Rides')
    axes[1].set_title('RMSE Distribution')
    axes[1].legend()
    
    # sMAPE distribution
    if len(val_data) > 0:
        axes[2].hist(val_data['smape'], bins=15, alpha=0.7, label='Validation', color=VAL_COLOR)
    if len(test_data) > 0:
        axes[2].hist(test_data['smape'], bins=15, alpha=0.7, label='Test', color=TEST_COLOR)
    axes[2].set_xlabel('sMAPE (%)')
    axes[2].set_ylabel('Number of Rides')
    axes[2].set_title('sMAPE Distribution')
    axes[2].legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "prophet_metrics_distribution_extended.png"))
    plt.close()
    
    # 3. Ride performance ranking (test set)
    if len(test_data) > 0:
        test_sorted = test_data.sort_values('mae')
        
        plt.figure(figsize=(16, 10))
        bars = plt.barh(range(len(test_sorted)), test_sorted['mae'], alpha=0.8, color=TEST_COLOR)
        
        plt.title('Prophet Model: Test Set MAE by Ride', fontsize=20, fontweight='bold')
        plt.xlabel('Mean Absolute Error (MAE) - Minutes', fontsize=16)
        plt.ylabel('Rides', fontsize=16)
        plt.yticks(range(len(test_sorted)), test_sorted['ride_name'], fontsize=14)
        
        # Add value labels
        for i, (bar, mae_val) in enumerate(zip(bars, test_sorted['mae'])):
            plt.text(bar.get_width() + max(test_sorted['mae']) * 0.01, 
                    bar.get_y() + bar.get_height()/2, 
                    f'{mae_val:.1f}', ha='left', va='center', fontsize=16)
        
        plt.grid(axis='x', alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "prophet_mae_by_ride_test.png"), dpi=300, bbox_inches='tight')
        plt.savefig(os.path.join(output_dir, "prophet_mae_by_ride_test.svg"), bbox_inches='tight')
        plt.close()
    
    print("Extended Prophet visualizations saved to output directory.")

def process_all_rides_extended(all_rides, train_data, val_data, test_data,
                              output_dir="models", resume=True):
    """Process all rides with Prophet models on both validation and test sets."""
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Get list of already processed rides
    processed_rides = []
    if resume:
        processed_rides = load_checkpoint_file(output_dir)
        if processed_rides:
            print(f"Resuming from checkpoint. {len(processed_rides)} rides already processed.")
    
    # Initialize results dictionary
    all_results = {}
    
    # Process each ride
    for i, ride_name in enumerate(tqdm(all_rides, desc="Processing rides")):
        if ride_name in processed_rides:
            print(f"Skipping {ride_name} (already processed)")
            # Load metrics for the summary
            _, metrics = load_model(ride_name, output_dir)
            if metrics:
                all_results[ride_name] = metrics
            continue
            
        print(f"\nProcessing ride {i+1}/{len(all_rides)}: {ride_name}")
        ride_metrics = process_single_ride_extended(ride_name, train_data, val_data, test_data,
                                                   output_dir=output_dir)
        
        if ride_metrics:
            all_results[ride_name] = ride_metrics
            processed_rides.append(ride_name)
            
            # Update checkpoint after each ride
            create_checkpoint_file(processed_rides, output_dir)
    
    # Generate summary report
    generate_summary_report_extended(all_results, output_dir)
    
    return all_results

## Analysis Functions

In [9]:
def analyze_prophet_results_extended(output_dir="../models/prophet_enhanced/"):
    """Load and analyze saved Prophet model results for both validation and test sets."""
    
    # Check if summary files exist
    detailed_path = os.path.join(output_dir, "detailed_prophet_summary_extended.csv")
    simple_path = os.path.join(output_dir, "prophet_summary_extended.csv")
    
    if not os.path.exists(detailed_path) or not os.path.exists(simple_path):
        print("Extended summary files not found. Run the processing pipeline first.")
        return None
    
    # Load summary data
    detailed_summary = pd.read_csv(detailed_path)
    simple_summary = pd.read_csv(simple_path)
    
    # Display key insights
    print("="*60)
    print("PROPHET MODEL ANALYSIS (VALIDATION & TEST SETS)")
    print("="*60)
    
    # Split by dataset
    val_data = detailed_summary[detailed_summary['dataset'] == 'validation']
    test_data = detailed_summary[detailed_summary['dataset'] == 'test']
    
    print("\n1. Overall Performance Summary:")
    print(simple_summary)
    
    if len(val_data) > 0:
        print("\n2. Validation Set - Top 10 rides (lowest MAE):")
        val_sorted = val_data.sort_values('mae')
        print(val_sorted.head(10)[['ride_name', 'mae', 'rmse', 'smape']])
    
    if len(test_data) > 0:
        print("\n3. Test Set - Top 10 rides (lowest MAE):")
        test_sorted = test_data.sort_values('mae')
        print(test_sorted.head(10)[['ride_name', 'mae', 'rmse', 'smape']])
    
    if len(val_data) > 0 and len(test_data) > 0:
        print("\n4. Validation vs Test Performance Comparison:")
        val_avg = val_data.groupby('dataset')[['mae', 'rmse', 'smape']].mean()
        test_avg = test_data.groupby('dataset')[['mae', 'rmse', 'smape']].mean()
        comparison_df = pd.concat([val_avg, test_avg])
        print(comparison_df)
    
    return {
        'detailed_summary': detailed_summary,
        'simple_summary': simple_summary,
        'val_data': val_data,
        'test_data': test_data
    }

def create_prophet_method_summary_csv(all_results, output_dir="../models/prophet_enhanced/", filename="prophet_method_summary.csv"):
    """
    Create a CSV file with Prophet model performance metrics in the same format as baselines.
    
    Args:
        all_results: Dictionary containing all Prophet model results
        output_dir: Directory to save the CSV file
        filename: Name of the output CSV file
    
    Returns:
        pandas.DataFrame: Summary dataframe
    """
    # Initialize data for Prophet
    val_mae_values = []
    val_rmse_values = []
    val_smape_values = []
    test_mae_values = []
    test_rmse_values = []
    test_smape_values = []
    
    # Collect metrics from all rides
    for ride_name, ride_results in all_results.items():
        if "validation" in ride_results:
            val_metrics = ride_results["validation"]
            if "mae" in val_metrics and not np.isinf(val_metrics["mae"]):
                val_mae_values.append(val_metrics["mae"])
            if "rmse" in val_metrics and not np.isinf(val_metrics["rmse"]):
                val_rmse_values.append(val_metrics["rmse"])
            if "smape" in val_metrics and not np.isinf(val_metrics["smape"]):
                val_smape_values.append(val_metrics["smape"])
        
        if "test" in ride_results:
            test_metrics = ride_results["test"]
            if "mae" in test_metrics and not np.isinf(test_metrics["mae"]):
                test_mae_values.append(test_metrics["mae"])
            if "rmse" in test_metrics and not np.isinf(test_metrics["rmse"]):
                test_rmse_values.append(test_metrics["rmse"])
            if "smape" in test_metrics and not np.isinf(test_metrics["smape"]):
                test_smape_values.append(test_metrics["smape"])
    
    # Calculate average metrics
    val_mae_avg = np.mean(val_mae_values) if val_mae_values else np.nan
    val_rmse_avg = np.mean(val_rmse_values) if val_rmse_values else np.nan
    val_smape_avg = np.mean(val_smape_values) if val_smape_values else np.nan
    
    test_mae_avg = np.mean(test_mae_values) if test_mae_values else np.nan
    test_rmse_avg = np.mean(test_rmse_values) if test_rmse_values else np.nan
    test_smape_avg = np.mean(test_smape_values) if test_smape_values else np.nan
    
    # Create summary data
    summary_data = [{
        'method_name': 'Prophet Enhanced',
        'val_mae': val_mae_avg,
        'val_rmse': val_rmse_avg,
        'val_smape': val_smape_avg,
        'test_mae': test_mae_avg,
        'test_rmse': test_rmse_avg,
        'test_smape': test_smape_avg
    }]
    
    # Create DataFrame
    summary_df = pd.DataFrame(summary_data)
    
    # Round values to 2 decimal places
    numeric_columns = ['val_mae', 'val_rmse', 'val_smape', 'test_mae', 'test_rmse', 'test_smape']
    for col in numeric_columns:
        summary_df[col] = summary_df[col].round(2)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save to CSV with semicolon separator
    output_path = os.path.join(output_dir, filename)
    summary_df.to_csv(output_path, sep=';', index=False)
    
    print(f"Prophet method summary CSV saved to: {output_path}")
    print("\nProphet model performance:")
    print(summary_df.to_string(index=False))
    
    return summary_df

## Main Execution

In [10]:
print("Loading data...")
data = load_data("../data/processed/ep/final_cleaned_processed_wait_times.parquet")
print(f"Loaded data with {len(data)} rows")

check_for_missing_values(data)

data = filter_to_operating_hours(data)

# Define time periods for splitting
train_years, val_year, test_year = list(range(2017, 2023)), 2023, 2024

# Split the data
train_data, val_data, test_data = split_data(data, train_years, val_year, test_year)

# Get all rides in the dataset
all_rides = get_all_rides(data)
print(f"Found {len(all_rides)} rides in the dataset:")
for i, ride in enumerate(all_rides):
    print(f"{i+1}. {ride}")

# Set output directory for models and results
output_dir = "../models/prophet_enhanced_extended/"

# Process all rides with extended evaluation (including test set)
print("\n" + "="*60)
print("STARTING PROPHET MODEL PROCESSING FOR ALL RIDES (VALIDATION & TEST SETS)")
print("="*60)

results = process_all_rides_extended(
    all_rides=all_rides,
    train_data=train_data,
    val_data=val_data,
    test_data=test_data,
    output_dir=output_dir,
    resume=True  # Resume from checkpoint if available
)

print("\n" + "="*60)
print("PROPHET MODEL PROCESSING COMPLETED")
print("="*60)

# Analyze the extended results
results_analysis = analyze_prophet_results_extended(output_dir)

# Create method summary CSV for comparison with baselines
create_prophet_method_summary_csv(results, output_dir)

Loading data...
Loaded data with 7834739 rows
No missing values found in the dataset.
Train data size: 297362
Validation data size: 61851
Test data size: 55699
Found 31 rides in the dataset:
1. alpine express enzian
2. arena of football  be part of it
3. arthur
4. atlantica supersplash
5. atlantis adventure
6. baaa express
7. blue fire megacoaster
8. castello dei medici
9. dancing dingie
10. euromir
11. eurosat  cancan coaster
12. eurotower
13. fjordrafting
14. jim button  journey through morrowland
15. josefinas magical imperial journey
16. kolumbusjolle
17. madame freudenreich curiosits
18. matterhornblitz
19. old mac donalds tractor fun
20. pegasus
21. poppy towers
22. poseidon
23. silver star
24. swiss bob run
25. tirol log flume
26. vienna wave swing  glckspilz
27. vindjammer
28. voletarium
29. volo da vinci
30. voltron nevera powered by rimac
31. whale adventures  northern lights

STARTING PROPHET MODEL PROCESSING FOR ALL RIDES (VALIDATION & TEST SETS)


Processing rides:   0%|          | 0/31 [00:00<?, ?it/s]


Processing ride 1/31: alpine express enzian

Processing ride: alpine express enzian
Training data size: 10302
Validation data size: 2019
Test data size: 1753
Training Prophet model on train data for validation evaluation...


18:38:45 - cmdstanpy - INFO - Chain [1] start processing
18:38:51 - cmdstanpy - INFO - Chain [1] done processing


Evaluating on validation set...

alpine express enzian - Validation MAE: 7.96 minutes
alpine express enzian - Validation RMSE: 9.96 minutes
alpine express enzian - Validation sMAPE: 28.06%
Training Prophet model on combined train + validation data for test evaluation...


...

Evaluating on test set...

whale adventures  northern lights - Test MAE: 1.26 minutes
whale adventures  northern lights - Test RMSE: 2.85 minutes
whale adventures  northern lights - Test sMAPE: 75.24%


Processing rides: 100%|██████████| 31/31 [08:39<00:00, 16.75s/it]

Models and results saved to ../models/prophet_enhanced_extended/whale_adventures__northern_lights

PROPHET MODEL SUMMARY (VALIDATION & TEST SETS):
Total rides processed: 30

Validation Set Performance:
  Average MAE: 7.10 ± 5.64
  Average RMSE: 9.16 ± 6.84
  Average sMAPE: 29.79% ± 19.66%

Test Set Performance:
  Average MAE: 9.24 ± 9.40
  Average RMSE: 11.87 ± 11.29
  Average sMAPE: 45.50% ± 23.69%

Detailed summary saved to: ../models/prophet_enhanced_extended/detailed_prophet_summary_extended.csv
Simple summary saved to: ../models/prophet_enhanced_extended/prophet_summary_extended.csv


Extended Prophet visualizations saved to output directory.

PROPHET MODEL PROCESSING COMPLETED
PROPHET MODEL ANALYSIS (VALIDATION & TEST SETS)

1. Overall Performance Summary:
      dataset  count  mae_mean   mae_std  rmse_mean   rmse_std  smape_mean  \
0  validation     30  7.104824  5.635081   9.159882   6.840383   29.794987   
1        test     30  9.235983  9.399306  11.865569  11.287848   45.495572   

   smape_std  
0  19.660159  
1  23.691822  

2. Validation Set - Top 10 rides (lowest MAE):
                            ride_name       mae      rmse      smape
32      madame freudenreich curiosits  0.067799  0.538258  99.403340
58  whale adventures  northern lights  0.938673  2.467813  92.219341
30                      kolumbusjolle  1.348201  1.886739  14.349242
36        old mac donalds tractor fun  1.886466  2.437464  38.950417
40                       poppy towers  1.933295  2.600554  21.541371
50       vienna wave swing  glckspilz  2.188961  2.688666  18.453290
52           

,method_name,val_mae,val_rmse,val_smape,test_mae,test_rmse,test_smape
0,Prophet Enhanced,7.1,9.16,29.79,9.24,11.87,45.5
